<a href="https://colab.research.google.com/github/ElisaVianey13/DSC/blob/main/KOMSTAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
####OUTLIER DENGAN KORELASI RENDAH###

import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm
import pprint

# ----------------------- Generate MV Normal + Outlier + Korelasi -----------------------
def generate_mvn_with_outliers(n, p, rho=0.15, outlier_ratio=0.01, outlier_magnitude=10):
    # Matriks kovarians dengan korelasi rho
    cov = rho * np.ones((p, p)) + (1 - rho) * np.eye(p)
    mean = np.zeros(p)

    # Data utama dari MV Normal
    X = np.random.multivariate_normal(mean, cov, size=n)

    # Tambahkan outlier
    n_outliers = int(outlier_ratio * n)
    if n_outliers > 0:
        idx = np.random.choice(n, n_outliers, replace=False)
        for i in idx:
            X[i] += outlier_magnitude * np.random.normal(size=p)

    return X

# ----------------------- Royston Test (Approximate via Shapiro) -----------------------
def royston_test(data):
    return np.mean([shapiro(data[:, i])[1] for i in range(data.shape[1])])

# ----------------------- Henze-Zirkler Statistic -----------------------
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X.T)
    Sinv = np.linalg.pinv(S)
    D2 = np.array([np.dot((x - mean_X), Sinv).dot((x - mean_X)) for x in X])
    hz = np.sum(np.exp(-0.5 * D2))
    return hz

# ----------------------- KTO Tests -----------------------
def kto_tests(data):
    n = data.shape[0]
    skew_vals = skew(data, axis=0)
    kurt_vals = kurtosis(data, axis=0, fisher=True)
    skew_z = np.sqrt(n) * skew_vals / np.sqrt(6)
    kurt_z = np.sqrt(n) * kurt_vals / np.sqrt(24)
    skew_p = 2 * (1 - norm.cdf(np.abs(skew_z).mean()))
    kurt_p = 2 * (1 - norm.cdf(np.abs(kurt_z).mean()))
    return skew_p, kurt_p

# ----------------------- Monte Carlo Simulasi -----------------------
def monte_carlo_mvn_outlier(p_values, n_values, iterations=1000, alphas=[0.01, 0.05]):
    results_all = {}

    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_skew': 0, 'KTO_kurt': 0} for alpha in alphas}

            # Threshold HZ dari data normal (tanpa outlier) sebagai batas kritis
            hz_null = [hz_statistic(np.random.multivariate_normal(np.zeros(p), np.eye(p), size=n)) for _ in range(iterations)]
            hz_thresholds = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}

            for _ in range(iterations):
                data = generate_mvn_with_outliers(n, p, rho=0.45, outlier_ratio=0.01, outlier_magnitude=10)
                hz_val = hz_statistic(data)
                roys_p = royston_test(data)
                skew_p, kurt_p = kto_tests(data)

                for alpha in alphas:
                    if hz_val > hz_thresholds[alpha]:
                        results[alpha]['HZ'] += 1
                    if roys_p < alpha:
                        results[alpha]['Royston'] += 1
                    if skew_p < alpha:
                        results[alpha]['KTO_skew'] += 1
                    if kurt_p < alpha:
                        results[alpha]['KTO_kurt'] += 1

            # Hitung proporsi penolakan
            for alpha in alphas:
                for test in results[alpha]:
                    results[alpha][test] /= iterations

            results_all[p][n] = results
    return results_all

# ----------------------- Parameter & Run -----------------------
p_values = [2, 3]
n_values = [20, 30, 100, 200]
alphas = [0.01, 0.05]

mvnorm_results = monte_carlo_mvn_outlier(p_values, n_values, iterations=1000, alphas=alphas)

# ----------------------- Tampilkan Hasil -----------------------
pprint.pprint(mvnorm_results)

{2: {20: {0.01: {'HZ': 0.009,
                 'KTO_kurt': 0.002,
                 'KTO_skew': 0.0,
                 'Royston': 0.0},
          0.05: {'HZ': 0.051,
                 'KTO_kurt': 0.005,
                 'KTO_skew': 0.009,
                 'Royston': 0.006}},
     30: {0.01: {'HZ': 0.009,
                 'KTO_kurt': 0.002,
                 'KTO_skew': 0.001,
                 'Royston': 0.0},
          0.05: {'HZ': 0.04,
                 'KTO_kurt': 0.005,
                 'KTO_skew': 0.005,
                 'Royston': 0.005}},
     100: {0.01: {'HZ': 0.794,
                  'KTO_kurt': 0.85,
                  'KTO_skew': 0.794,
                  'Royston': 0.444},
           0.05: {'HZ': 0.854,
                  'KTO_kurt': 0.868,
                  'KTO_skew': 0.833,
                  'Royston': 0.511}},
     200: {0.01: {'HZ': 0.972,
                  'KTO_kurt': 0.984,
                  'KTO_skew': 0.947,
                  'Royston': 0.808},
           0.05: {'HZ': 0.9

In [ ]:
########MV LOG NORMAL TANPA OUTLIER############

import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm
import pprint

# Fungsi generate data Multivariate Log-Normal tanpa outlier
def generate_mvlognormal(n, p):
    mean = np.zeros(p)
    cov = np.eye(p)  # tanpa korelasi
    normal_data = np.random.multivariate_normal(mean, cov, size=n)
    lognormal_data = np.exp(normal_data)  # transformasi log-normal
    return lognormal_data

# Approximate Royston test: rata-rata p-value dari Shapiro-Wilk
def royston_test(data):
    return np.mean([shapiro(data[:, i])[1] for i in range(data.shape[1])])

# Statistik HZ (approximate)
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X.T)
    Sinv = np.linalg.pinv(S)
    D2 = np.array([np.dot((x - mean_X), Sinv).dot((x - mean_X)) for x in X])
    hz = np.sum(np.exp(-0.5 * D2))
    return hz

# Uji KTO untuk skewness dan kurtosis
def kto_tests(data):
    n = data.shape[0]
    skew_vals = skew(data, axis=0)
    kurt_vals = kurtosis(data, axis=0, fisher=True)
    skew_z = np.sqrt(n) * skew_vals / np.sqrt(6)
    kurt_z = np.sqrt(n) * kurt_vals / np.sqrt(24)
    skew_p = 2 * (1 - norm.cdf(np.abs(skew_z).mean()))
    kurt_p = 2 * (1 - norm.cdf(np.abs(kurt_z).mean()))
    return skew_p, kurt_p

# Fungsi simulasi Monte Carlo
def monte_carlo_mvlognormal(p_values, n_values, iterations=1000, alphas=[0.01, 0.05]):
    results_all = {}

    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_skew': 0, 'KTO_kurt': 0} for alpha in alphas}

            # Threshold HZ dari data normal
            hz_null = [hz_statistic(np.random.multivariate_normal(np.zeros(p), np.eye(p), size=n)) for _ in range(iterations)]
            hz_thresholds = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}

            for _ in range(iterations):
                data = generate_mvlognormal(n, p)
                hz_val = hz_statistic(data)
                roys_p = royston_test(data)
                skew_p, kurt_p = kto_tests(data)

                for alpha in alphas:
                    if hz_val > hz_thresholds[alpha]:
                        results[alpha]['HZ'] += 1
                    if roys_p < alpha:
                        results[alpha]['Royston'] += 1
                    if skew_p < alpha:
                        results[alpha]['KTO_skew'] += 1
                    if kurt_p < alpha:
                        results[alpha]['KTO_kurt'] += 1

            # Estimasi proporsi (Type I error / power)
            for alpha in alphas:
                for test in results[alpha]:
                    results[alpha][test] /= iterations

            results_all[p][n] = results
    return results_all

# Parameter simulasi
p_values = [2, 3]
n_values = [20, 30, 100, 200]
alphas = [0.01, 0.05]

# Jalankan simulasi
mvlog_results = monte_carlo_mvlognormal(p_values, n_values, iterations=1000, alphas=alphas)

# Cetak hasil
pprint.pprint(mvlog_results)


{2: {20: {0.01: {'HZ': 0.673,
                 'KTO_kurt': 0.518,
                 'KTO_skew': 0.75,
                 'Royston': 0.744},
          0.05: {'HZ': 0.762,
                 'KTO_kurt': 0.642,
                 'KTO_skew': 0.903,
                 'Royston': 0.919}},
     30: {0.01: {'HZ': 0.841,
                 'KTO_kurt': 0.809,
                 'KTO_skew': 0.961,
                 'Royston': 0.957},
          0.05: {'HZ': 0.881,
                 'KTO_kurt': 0.871,
                 'KTO_skew': 0.991,
                 'Royston': 0.99}},
     100: {0.01: {'HZ': 1.0, 'KTO_kurt': 1.0, 'KTO_skew': 1.0, 'Royston': 1.0},
           0.05: {'HZ': 1.0, 'KTO_kurt': 1.0, 'KTO_skew': 1.0, 'Royston': 1.0}},
     200: {0.01: {'HZ': 1.0, 'KTO_kurt': 1.0, 'KTO_skew': 1.0, 'Royston': 1.0},
           0.05: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 1.0,
                  'Royston': 1.0}}},
 3: {20: {0.01: {'HZ': 0.699,
                 'KTO_kurt': 0.608,
     

In [ ]:
import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm
import pprint

# --------- Generate ambiguous data: sedikit "tidak normal" ----------
def generate_ambiguous_data(n, p, rho=0.45, contam_prop=0.05):
    cov = (1 - rho) * np.eye(p) + rho * np.ones((p, p))
    data = np.random.multivariate_normal(np.zeros(p), cov, size=n)
    n_contam = int(n * contam_prop)
    if n_contam > 0:
        contamination = np.random.normal(loc=5, scale=1, size=(n_contam, p))
        data[:n_contam] += contamination
    return data

# --------- Royston test (mean of Shapiro-Wilk p-values) ----------
def royston_test(data):
    return np.mean([shapiro(data[:, i])[1] for i in range(data.shape[1])])

# --------- HZ statistic ----------
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X.T)
    Sinv = np.linalg.pinv(S)
    D2 = np.array([np.dot((x - mean_X), Sinv).dot((x - mean_X)) for x in X])
    hz = np.sum(np.exp(-0.5 * D2))
    return hz

# --------- KTO Tests ----------
def kto_tests(data):
    n = data.shape[0]
    skew_vals = skew(data, axis=0)
    kurt_vals = kurtosis(data, axis=0, fisher=True)
    skew_z = np.sqrt(n) * skew_vals / np.sqrt(6)
    kurt_z = np.sqrt(n) * kurt_vals / np.sqrt(24)
    skew_p = 2 * (1 - norm.cdf(np.abs(skew_z).mean()))
    kurt_p = 2 * (1 - norm.cdf(np.abs(kurt_z).mean()))
    return skew_p, kurt_p

# --------- Monte Carlo Simulation ----------
def monte_carlo_sim(p_values, n_values, iterations=1000, alphas=[0.01, 0.05], rho=0.45):
    results_all = {}

    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_skew': 0, 'KTO_kurt': 0} for alpha in alphas}

            # Threshold untuk HZ berdasarkan distribusi normal
            hz_null = [hz_statistic(generate_ambiguous_data(n, p, rho=rho, contam_prop=0)) for _ in range(iterations)]
            hz_thresholds = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}

            for _ in range(iterations):
                data = generate_ambiguous_data(n, p, rho=rho, contam_prop=0.05)  # ambiguous!
                hz_val = hz_statistic(data)
                roys_p = royston_test(data)
                skew_p, kurt_p = kto_tests(data)

                for alpha in alphas:
                    if hz_val > hz_thresholds[alpha]:
                        results[alpha]['HZ'] += 1
                    if roys_p < alpha:
                        results[alpha]['Royston'] += 1
                    if skew_p < alpha:
                        results[alpha]['KTO_skew'] += 1
                    if kurt_p < alpha:
                        results[alpha]['KTO_kurt'] += 1

            # Ubah jadi proporsi
            for alpha in alphas:
                for test in results[alpha]:
                    results[alpha][test] /= iterations

            results_all[p][n] = results

    return results_all

# --------- Parameter Simulasi ----------
p_values = [2, 3]
n_values = [20, 30, 100, 200]
alphas = [0.01, 0.05]

# --------- Jalankan Simulasi ----------
hasil = monte_carlo_sim(p_values, n_values, iterations=1000, alphas=alphas)

# --------- Tampilkan hasil ---------
pprint.pprint(hasil)


{2: {20: {0.01: {'HZ': 0.394,
                 'KTO_kurt': 0.643,
                 'KTO_skew': 0.586,
                 'Royston': 0.409},
          0.05: {'HZ': 0.657,
                 'KTO_kurt': 0.778,
                 'KTO_skew': 0.778,
                 'Royston': 0.623}},
     30: {0.01: {'HZ': 0.487,
                 'KTO_kurt': 0.792,
                 'KTO_skew': 0.697,
                 'Royston': 0.495},
          0.05: {'HZ': 0.717,
                 'KTO_kurt': 0.853,
                 'KTO_skew': 0.849,
                 'Royston': 0.662}},
     100: {0.01: {'HZ': 0.993,
                  'KTO_kurt': 0.999,
                  'KTO_skew': 1.0,
                  'Royston': 1.0},
           0.05: {'HZ': 0.999,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 1.0,
                  'Royston': 1.0}},
     200: {0.01: {'HZ': 1.0, 'KTO_kurt': 1.0, 'KTO_skew': 1.0, 'Royston': 1.0},
           0.05: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew

In [ ]:
#######Tanpa outlier, korelasi rendah (0,15)#####

import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm, chi2
import pprint

# ---------------------- Generate Data ----------------------
def generate_data(n, p, rho=0.15):
    cov = (1 - rho) * np.eye(p) + rho * np.ones((p, p))
    data = np.random.multivariate_normal(np.zeros(p), cov, size=n)
    return data

# ---------------------- Royston Test (Kolmogorov Type) ----------------------
def royston_test(data):
    n, p = data.shape
    max_diff_list = []

    for j in range(p):
        x = np.sort(data[:, j])
        F_empirical = np.arange(1, n + 1) / n
        F_theoretical = norm.cdf(x, loc=np.mean(x), scale=np.std(x, ddof=1))
        diffs = np.abs(F_empirical - F_theoretical)
        max_diff = np.max(diffs)
        max_diff_list.append(max_diff)

    Z = max(max_diff_list)
    return Z

# ---------------------- Henze-Zirkler Test ----------------------
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X, rowvar=False)
    Sinv = np.linalg.pinv(S)

    def mahalanobis_sq(x, y):
        delta = x - y
        return delta.T @ Sinv @ delta

    D2 = np.array([[mahalanobis_sq(X[i], X[j]) for j in range(n)] for i in range(n)])
    beta = 1.0 / np.sqrt(2.0)
    A = np.sum(np.exp(-beta*2 / 2 * D2)) / n**2
    B = np.sum(np.exp(-beta*2 / (2 * (1 + beta*2)) * np.diag(D2))) * 2 / n
    C = (1 + 2 * beta**2) ** (-p / 2)
    hz = A - B + C
    return hz

# ---------------------- Optimized KTO Test ----------------------
def kto_test_fast(data):
    n, p = data.shape
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    Sinv = np.linalg.pinv(cov)

    centered = data - mean
    r2 = np.sum(centered @ Sinv * centered, axis=1)

    diff = centered[:, None, :] - centered[None, :, :]
    rij = np.einsum('ijk,kl,ijl->ij', diff, Sinv, diff)

    b1_new = np.mean(r2) - 2 * np.mean(r2[:, None] * r2[None, :] * rij)

    term1 = np.mean(r2[:, None] * r2[None, :] * rij**2)
    term2 = 2 * np.mean(r2**2)
    b2_new = ((p + 2)**-2) * term1 - ((p + 2)**-1) * term2 + p

    return b1_new, b2_new

# ---------------------- Monte Carlo Simulation ----------------------
def monte_carlo_sim(p_values, n_values, iterations=100, alphas=[0.05], rho=0.15):
    results_all = {}
    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_b1': 0, 'KTO_b2': 0} for alpha in alphas}

            # Thresholds from null distribution
            hz_null = [hz_statistic(generate_data(n, p, rho)) for _ in range(iterations)]
            royston_null = [royston_test(generate_data(n, p, rho)) for _ in range(iterations)]
            kto_b1_null = [kto_test_fast(generate_data(n, p, rho))[0] for _ in range(iterations)]
            kto_b2_null = [kto_test_fast(generate_data(n, p, rho))[1] for _ in range(iterations)]

            hz_thresh = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}
            royston_thresh = {alpha: np.percentile(royston_null, 100 * (1 - alpha)) for alpha in alphas}
            b1_thresh = {alpha: np.percentile(kto_b1_null, 100 * alpha) for alpha in alphas}
            b2_thresh = {alpha: np.percentile(kto_b2_null, 100 * alpha) for alpha in alphas}

            for _ in range(iterations):
                data = generate_data(n, p, rho)
                hz = hz_statistic(data)
                royston = royston_test(data)
                b1, b2 = kto_test_fast(data)

                for alpha in alphas:
                    if hz > hz_thresh[alpha]:
                        results[alpha]['HZ'] += 1
                    if royston > royston_thresh[alpha]:
                        results[alpha]['Royston'] += 1
                    if b1 < b1_thresh[alpha]:
                        results[alpha]['KTO_b1'] += 1
                    if b2 < b2_thresh[alpha]:
                        results[alpha]['KTO_b2'] += 1

            for alpha in alphas:
                for key in results[alpha]:
                    results[alpha][key] /= iterations

            results_all[p][n] = results
    return results_all

# ---------------------- Eksekusi ----------------------
p_values = [2, 3]
n_values = [20, 30, 100, 200]
alphas = [0.01, 0.05]

type1_error_results = monte_carlo_sim(p_values, n_values, iterations=1000, alphas=alphas, rho=0.15)
pprint.pprint(type1_error_results)

{2: {20: {0.01: {'HZ': 0.0, 'KTO_b1': 0.005, 'KTO_b2': 0.017, 'Royston': 0.014},
          0.05: {'HZ': 0.033,
                 'KTO_b1': 0.046,
                 'KTO_b2': 0.039,
                 'Royston': 0.059}},
     30: {0.01: {'HZ': 0.011,
                 'KTO_b1': 0.012,
                 'KTO_b2': 0.008,
                 'Royston': 0.008},
          0.05: {'HZ': 0.049,
                 'KTO_b1': 0.058,
                 'KTO_b2': 0.043,
                 'Royston': 0.057}},
     100: {0.01: {'HZ': 0.006,
                  'KTO_b1': 0.009,
                  'KTO_b2': 0.021,
                  'Royston': 0.01},
           0.05: {'HZ': 0.057,
                  'KTO_b1': 0.052,
                  'KTO_b2': 0.068,
                  'Royston': 0.047}},
     200: {0.01: {'HZ': 0.005,
                  'KTO_b1': 0.005,
                  'KTO_b2': 0.005,
                  'Royston': 0.008},
           0.05: {'HZ': 0.042,
                  'KTO_b1': 0.042,
                  'KTO_b2': 0.053,


In [ ]:
######KORELASI RENDAH (0.15) DENGAN OUTLIER####

import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm, chi2
import pprint

# ---------------------- Generate Data with Outliers ----------------------
def generate_data(n, p, rho=0.15, outlier_ratio=0.01):
    """
    Menghasilkan data normal multivariat dengan tambahan outlier.

    Parameters:
        n (int): Jumlah baris (observasi)
        p (int): Jumlah kolom (variabel)
        rho (float): Korelasi antar variabel
        outlier_ratio (float): Proporsi total elemen sebagai outlier (misalnya 0.01 = 1%)

    Returns:
        np.array: Data berdimensi (n x p) dengan beberapa outlier
    """
    # Matriks kovarians
    cov = (1 - rho) * np.eye(p) + rho * np.ones((p, p))
    data = np.random.multivariate_normal(np.zeros(p), cov, size=n)

    # Tambahkan outlier
    num_outliers = int(outlier_ratio * n * p)
    if num_outliers > 0:
        indices = np.random.choice(n * p, size=num_outliers, replace=False)
        std = np.std(data)
        for idx in indices:
            row = idx // p
            col = idx % p
            data[row, col] += np.random.choice([-1, 1]) * 10 * std  # Outlier ±10 SD

    return data


# ---------------------- Royston Test (Kolmogorov Type) ----------------------
def royston_test(data):
    n, p = data.shape
    max_diff_list = []

    for j in range(p):
        x = np.sort(data[:, j])
        F_empirical = np.arange(1, n + 1) / n
        F_theoretical = norm.cdf(x, loc=np.mean(x), scale=np.std(x, ddof=1))
        diffs = np.abs(F_empirical - F_theoretical)
        max_diff = np.max(diffs)
        max_diff_list.append(max_diff)

    Z = max(max_diff_list)
    return Z


# ---------------------- Henze-Zirkler Test ----------------------
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X, rowvar=False)
    Sinv = np.linalg.pinv(S)

    def mahalanobis_sq(x, y):
        delta = x - y
        return delta.T @ Sinv @ delta

    D2 = np.array([[mahalanobis_sq(X[i], X[j]) for j in range(n)] for i in range(n)])
    beta = 1.0 / np.sqrt(2.0)
    A = np.sum(np.exp(-beta*2 / 2 * D2)) / n**2
    B = np.sum(np.exp(-beta*2 / (2 * (1 + beta*2)) * np.diag(D2))) * 2 / n
    C = (1 + 2 * beta**2) ** (-p / 2)
    hz = A - B + C
    return hz


# ---------------------- Optimized KTO Test ----------------------
def kto_test_fast(data):
    n, p = data.shape
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    Sinv = np.linalg.pinv(cov)

    centered = data - mean
    r2 = np.sum(centered @ Sinv * centered, axis=1)

    diff = centered[:, None, :] - centered[None, :, :]
    rij = np.einsum('ijk,kl,ijl->ij', diff, Sinv, diff)

    b1_new = np.mean(r2) - 2 * np.mean(r2[:, None] * r2[None, :] * rij)

    term1 = np.mean(r2[:, None] * r2[None, :] * rij**2)
    term2 = 2 * np.mean(r2**2)
    b2_new = ((p + 2)**-2) * term1 - ((p + 2)**-1) * term2 + p

    return b1_new, b2_new


# ---------------------- Monte Carlo Simulation ----------------------
def monte_carlo_sim(p_values, n_values, iterations=1000, alphas=[0.05], rho=0.15, outlier_ratio=0.01):
    results_all = {}
    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_b1': 0, 'KTO_b2': 0} for alpha in alphas}

            # Thresholds from null distribution
            hz_null = [hz_statistic(generate_data(n, p, rho, outlier_ratio)) for _ in range(iterations)]
            royston_null = [royston_test(generate_data(n, p, rho, outlier_ratio)) for _ in range(iterations)]
            kto_b1_null = [kto_test_fast(generate_data(n, p, rho, outlier_ratio))[0] for _ in range(iterations)]
            kto_b2_null = [kto_test_fast(generate_data(n, p, rho, outlier_ratio))[1] for _ in range(iterations)]

            hz_thresh = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}
            royston_thresh = {alpha: np.percentile(royston_null, 100 * (1 - alpha)) for alpha in alphas}
            b1_thresh = {alpha: np.percentile(kto_b1_null, 100 * alpha) for alpha in alphas}
            b2_thresh = {alpha: np.percentile(kto_b2_null, 100 * alpha) for alpha in alphas}

            for _ in range(iterations):
                data = generate_data(n, p, rho, outlier_ratio)
                hz = hz_statistic(data)
                royston = royston_test(data)
                b1, b2 = kto_test_fast(data)

                for alpha in alphas:
                    if hz > hz_thresh[alpha]:
                        results[alpha]['HZ'] += 1
                    if royston > royston_thresh[alpha]:
                        results[alpha]['Royston'] += 1
                    if b1 < b1_thresh[alpha]:
                        results[alpha]['KTO_b1'] += 1
                    if b2 < b2_thresh[alpha]:
                        results[alpha]['KTO_b2'] += 1

            for alpha in alphas:
                for key in results[alpha]:
                    results[alpha][key] /= iterations

            results_all[p][n] = results
    return results_all


# ---------------------- Eksekusi ----------------------
if __name__ == "__main__":
    p_values = [2, 3]
    n_values = [20, 30, 100, 200]
    alphas = [0.01, 0.05]

    power = monte_carlo_sim(
        p_values=p_values,
        n_values=n_values,
        iterations=1000,
        alphas=alphas,
        rho=0.15,
        outlier_ratio=0.01
    )

    pprint.pprint(power)

{2: {20: {0.01: {'HZ': 0.012,
                 'KTO_b1': 0.007,
                 'KTO_b2': 0.02,
                 'Royston': 0.024},
          0.05: {'HZ': 0.049,
                 'KTO_b1': 0.051,
                 'KTO_b2': 0.071,
                 'Royston': 0.057}},
     30: {0.01: {'HZ': 0.016,
                 'KTO_b1': 0.01,
                 'KTO_b2': 0.015,
                 'Royston': 0.015},
          0.05: {'HZ': 0.071,
                 'KTO_b1': 0.067,
                 'KTO_b2': 0.051,
                 'Royston': 0.05}},
     100: {0.01: {'HZ': 0.014,
                  'KTO_b1': 0.009,
                  'KTO_b2': 0.012,
                  'Royston': 0.009},
           0.05: {'HZ': 0.059,
                  'KTO_b1': 0.041,
                  'KTO_b2': 0.075,
                  'Royston': 0.046}},
     200: {0.01: {'HZ': 0.007,
                  'KTO_b1': 0.005,
                  'KTO_b2': 0.015,
                  'Royston': 0.012},
           0.05: {'HZ': 0.057,
                  '

In [16]:
######MV Log-Normal DENGAN outlier#####

import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm
import pprint

# Fungsi generate data MV Log-Normal DENGAN outlier 1%
def generate_mvlognormal_outlier(n, p, outlier_prop=0.01, magnitude=10):
    mean = np.zeros(p)
    cov = np.eye(p)
    normal_data = np.random.multivariate_normal(mean, cov, size=n)
    lognormal_data = np.exp(normal_data)

    # Tambahkan outlier
    n_outliers = int(n * outlier_prop)
    if n_outliers > 0:
        idx = np.random.choice(n, n_outliers, replace=False)
        for i in idx:
            lognormal_data[i] += magnitude * np.random.normal(size=p)

    return lognormal_data

# Royston test (approximate dengan Shapiro rata-rata)
def royston_test(data):
    return np.mean([shapiro(data[:, i])[1] for i in range(data.shape[1])])

# HZ Statistic (approximate)
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X.T)
    Sinv = np.linalg.pinv(S)
    D2 = np.array([np.dot((x - mean_X), Sinv).dot((x - mean_X)) for x in X])
    hz = np.sum(np.exp(-0.5 * D2))
    return hz

# KTO test
def kto_tests(data):
    n = data.shape[0]
    skew_vals = skew(data, axis=0)
    kurt_vals = kurtosis(data, axis=0, fisher=True)
    skew_z = np.sqrt(n) * skew_vals / np.sqrt(6)
    kurt_z = np.sqrt(n) * kurt_vals / np.sqrt(24)
    skew_p = 2 * (1 - norm.cdf(np.abs(skew_z).mean()))
    kurt_p = 2 * (1 - norm.cdf(np.abs(kurt_z).mean()))
    return skew_p, kurt_p

# Simulasi Monte Carlo dengan outlier
def monte_carlo_mvlognormal_outlier(p_values, n_values, iterations=1000, alphas=[0.01, 0.05], outlier_prop=0.01):
    results_all = {}

    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_skew': 0, 'KTO_kurt': 0} for alpha in alphas}

            # Threshold HZ dari data normal
            hz_null = [hz_statistic(np.random.multivariate_normal(np.zeros(p), np.eye(p), size=n)) for _ in range(iterations)]
            hz_thresholds = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}

            for _ in range(iterations):
                data = generate_mvlognormal_outlier(n, p, outlier_prop=outlier_prop, magnitude=10)
                hz_val = hz_statistic(data)
                roys_p = royston_test(data)
                skew_p, kurt_p = kto_tests(data)

                for alpha in alphas:
                    if hz_val > hz_thresholds[alpha]:
                        results[alpha]['HZ'] += 1
                    if roys_p < alpha:
                        results[alpha]['Royston'] += 1
                    if skew_p < alpha:
                        results[alpha]['KTO_skew'] += 1
                    if kurt_p < alpha:
                        results[alpha]['KTO_kurt'] += 1

            # Hitung proporsi penolakan
            for alpha in alphas:
                for test in results[alpha]:
                    results[alpha][test] /= iterations

            results_all[p][n] = results
    return results_all

# Parameter
p_values = [2, 3]
n_values = [20, 30, 100, 200]
alphas = [0.01, 0.05]

# Jalankan simulasi
mvlog_results_outlier = monte_carlo_mvlognormal_outlier(
    p_values, n_values, iterations=1000, alphas=alphas, outlier_prop=0.01
)

# Tampilkan hasil
pprint.pprint(mvlog_results_outlier)


{2: {20: {0.01: {'HZ': 0.615,
                 'KTO_kurt': 0.541,
                 'KTO_skew': 0.739,
                 'Royston': 0.76},
          0.05: {'HZ': 0.773,
                 'KTO_kurt': 0.649,
                 'KTO_skew': 0.913,
                 'Royston': 0.919}},
     30: {0.01: {'HZ': 0.8,
                 'KTO_kurt': 0.794,
                 'KTO_skew': 0.965,
                 'Royston': 0.952},
          0.05: {'HZ': 0.894,
                 'KTO_kurt': 0.866,
                 'KTO_skew': 0.994,
                 'Royston': 0.99}},
     100: {0.01: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 0.996,
                  'Royston': 1.0},
           0.05: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 0.998,
                  'Royston': 1.0}},
     200: {0.01: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 0.996,
                  'Royston': 1.0},
           0.05: {'HZ': 1.0,
            

In [21]:
import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm
import pprint

# Fungsi untuk membangkitkan data normal multivariat dengan korelasi + outlier
def generate_high_corr_normal(n, p, rho=0.15, prop_outlier=0.01):
    # Matriks kovarian dengan korelasi rho
    cov = np.full((p, p), rho)
    np.fill_diagonal(cov, 1.0)
    mean = np.zeros(p)

    data = np.random.multivariate_normal(mean, cov, size=n)

    # Tambahkan outlier 1%
    if prop_outlier > 0:
        n_outlier = int(np.floor(prop_outlier * n))
        if n_outlier > 0:
            outliers = np.random.multivariate_normal(mean=np.ones(p) * 10, cov=np.eye(p) * 5, size=n_outlier)
            data[:n_outlier] = outliers

    return data

# Approximate Royston test
def royston_test(data):
    return np.mean([shapiro(data[:, i])[1] for i in range(data.shape[1])])

# Henze-Zirkler approximate statistic
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X.T)
    Sinv = np.linalg.pinv(S)
    D2 = np.array([np.dot((x - mean_X), Sinv).dot((x - mean_X)) for x in X])
    hz = np.sum(np.exp(-0.5 * D2))
    return hz

# KTO Skewness dan Kurtosis test
def kto_tests(data):
    n = data.shape[0]
    skew_vals = skew(data, axis=0)
    kurt_vals = kurtosis(data, axis=0, fisher=True)
    skew_z = np.sqrt(n) * skew_vals / np.sqrt(6)
    kurt_z = np.sqrt(n) * kurt_vals / np.sqrt(24)
    skew_p = 2 * (1 - norm.cdf(np.abs(skew_z).mean()))
    kurt_p = 2 * (1 - norm.cdf(np.abs(kurt_z).mean()))
    return skew_p, kurt_p

# Fungsi simulasi Monte Carlo
def monte_carlo_corr_normal(p_values, n_values, rho=0.15, iterations=1000, alphas=[0.01, 0.05], prop_outlier=0.01):
    results_all = {}

    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_skew': 0, 'KTO_kurt': 0} for alpha in alphas}

            # Matriks kovarian untuk threshold HZ dari data normal tanpa outlier
            cov = np.full((p, p), rho)
            np.fill_diagonal(cov, 1.0)

            hz_null = [hz_statistic(np.random.multivariate_normal(np.zeros(p), cov, size=n)) for _ in range(iterations)]
            hz_thresholds = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}

            for _ in range(iterations):
                data = generate_high_corr_normal(n, p, rho=rho, prop_outlier=prop_outlier)
                hz_val = hz_statistic(data)
                roys_p = royston_test(data)
                skew_p, kurt_p = kto_tests(data)

                for alpha in alphas:
                    if hz_val > hz_thresholds[alpha]:
                        results[alpha]['HZ'] += 1
                    if roys_p < alpha:
                        results[alpha]['Royston'] += 1
                    if skew_p < alpha:
                        results[alpha]['KTO_skew'] += 1
                    if kurt_p < alpha:
                        results[alpha]['KTO_kurt'] += 1

            for alpha in alphas:
                for test in results[alpha]:
                    results[alpha][test] /= iterations

            results_all[p][n] = results
    return results_all

# Parameter simulasi
p_values = [2, 3]
n_values = [20, 30, 100, 200]
alphas = [0.01, 0.05]
prop_outlier = 0.01   # 1% outlier
rho_value = 0.5

# Jalankan simulasi
corr_outlier_results = monte_carlo_corr_normal(
    p_values, n_values, rho=rho_value, iterations=1000, alphas=alphas, prop_outlier=prop_outlier)

# Cetak hasil
pprint.pprint(corr_outlier_results)

{2: {20: {0.01: {'HZ': 0.005,
                 'KTO_kurt': 0.001,
                 'KTO_skew': 0.0,
                 'Royston': 0.0},
          0.05: {'HZ': 0.04,
                 'KTO_kurt': 0.006,
                 'KTO_skew': 0.006,
                 'Royston': 0.004}},
     30: {0.01: {'HZ': 0.003,
                 'KTO_kurt': 0.001,
                 'KTO_skew': 0.0,
                 'Royston': 0.0},
          0.05: {'HZ': 0.033,
                 'KTO_kurt': 0.005,
                 'KTO_skew': 0.005,
                 'Royston': 0.004}},
     100: {0.01: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 1.0,
                  'Royston': 0.984},
           0.05: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 1.0,
                  'Royston': 0.985}},
     200: {0.01: {'HZ': 1.0, 'KTO_kurt': 1.0, 'KTO_skew': 1.0, 'Royston': 1.0},
           0.05: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 1.0,
   

In [22]:
####KORELASI 0,15 DENGAN OUTLIER FIXXX#####
import numpy as np
from scipy.stats import kurtosis, skew, shapiro, norm
import pprint

# Fungsi untuk membangkitkan data normal multivariat dengan korelasi + outlier
def generate_corr_normal(n, p, rho=0.15, prop_outlier=0.01):
    # Matriks kovarian dengan korelasi rho
    cov = np.full((p, p), rho)
    np.fill_diagonal(cov, 1.0)
    mean = np.zeros(p)

    data = np.random.multivariate_normal(mean, cov, size=n)

    # Tambahkan outlier 1%
    if prop_outlier > 0:
        n_outlier = int(np.floor(prop_outlier * n))
        if n_outlier > 0:
            outliers = np.random.multivariate_normal(mean=np.ones(p) * 10, cov=np.eye(p) * 5, size=n_outlier)
            data[:n_outlier] = outliers

    return data

# Approximate Royston test
def royston_test(data):
    return np.mean([shapiro(data[:, i])[1] for i in range(data.shape[1])])

# Henze-Zirkler approximate statistic
def hz_statistic(X):
    n, p = X.shape
    mean_X = np.mean(X, axis=0)
    S = np.cov(X.T)
    Sinv = np.linalg.pinv(S)
    D2 = np.array([np.dot((x - mean_X), Sinv).dot((x - mean_X)) for x in X])
    hz = np.sum(np.exp(-0.5 * D2))
    return hz

# KTO Skewness dan Kurtosis test
def kto_tests(data):
    n = data.shape[0]
    skew_vals = skew(data, axis=0)
    kurt_vals = kurtosis(data, axis=0, fisher=True)
    skew_z = np.sqrt(n) * skew_vals / np.sqrt(6)
    kurt_z = np.sqrt(n) * kurt_vals / np.sqrt(24)
    skew_p = 2 * (1 - norm.cdf(np.abs(skew_z).mean()))
    kurt_p = 2 * (1 - norm.cdf(np.abs(kurt_z).mean()))
    return skew_p, kurt_p

# Fungsi simulasi Monte Carlo
def monte_carlo_corr_normal(p_values, n_values, rho=0.85, iterations=1000, alphas=[0.01, 0.05], prop_outlier=0.01):
    results_all = {}

    for p in p_values:
        results_all[p] = {}
        for n in n_values:
            results = {alpha: {'HZ': 0, 'Royston': 0, 'KTO_skew': 0, 'KTO_kurt': 0} for alpha in alphas}

            # Matriks kovarian untuk threshold HZ dari data normal tanpa outlier
            cov = np.full((p, p), rho)
            np.fill_diagonal(cov, 1.0)

            hz_null = [hz_statistic(np.random.multivariate_normal(np.zeros(p), cov, size=n)) for _ in range(iterations)]
            hz_thresholds = {alpha: np.percentile(hz_null, 100 * (1 - alpha)) for alpha in alphas}

            for _ in range(iterations):
                data = generate_corr_normal(n, p, rho=rho, prop_outlier=prop_outlier)
                hz_val = hz_statistic(data)
                roys_p = royston_test(data)
                skew_p, kurt_p = kto_tests(data)

                for alpha in alphas:
                    if hz_val > hz_thresholds[alpha]:
                        results[alpha]['HZ'] += 1
                    if roys_p < alpha:
                        results[alpha]['Royston'] += 1
                    if skew_p < alpha:
                        results[alpha]['KTO_skew'] += 1
                    if kurt_p < alpha:
                        results[alpha]['KTO_kurt'] += 1

            for alpha in alphas:
                for test in results[alpha]:
                    results[alpha][test] /= iterations

            results_all[p][n] = results
    return results_all

# Parameter simulasi
p_values = [2, 3]
n_values = [20, 30, 100, 200]
alphas = [0.01, 0.05]
prop_outlier = 0.01   # 1% outlier
rho_value = 0.15

# Jalankan simulasi
corr_outlier_results = monte_carlo_corr_normal(
    p_values, n_values, rho=rho_value, iterations=1000, alphas=alphas, prop_outlier=prop_outlier)

# Cetak hasil
pprint.pprint(corr_outlier_results)

{2: {20: {0.01: {'HZ': 0.009,
                 'KTO_kurt': 0.001,
                 'KTO_skew': 0.0,
                 'Royston': 0.0},
          0.05: {'HZ': 0.049,
                 'KTO_kurt': 0.002,
                 'KTO_skew': 0.007,
                 'Royston': 0.005}},
     30: {0.01: {'HZ': 0.006,
                 'KTO_kurt': 0.003,
                 'KTO_skew': 0.0,
                 'Royston': 0.001},
          0.05: {'HZ': 0.057,
                 'KTO_kurt': 0.006,
                 'KTO_skew': 0.006,
                 'Royston': 0.001}},
     100: {0.01: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 0.999,
                  'Royston': 0.985},
           0.05: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 1.0,
                  'Royston': 0.989}},
     200: {0.01: {'HZ': 1.0, 'KTO_kurt': 1.0, 'KTO_skew': 1.0, 'Royston': 1.0},
           0.05: {'HZ': 1.0,
                  'KTO_kurt': 1.0,
                  'KTO_skew': 1.0